In [4]:
import sys
import json
import time
from collections import defaultdict
import pandas as pd
import os
import numpy as np

sys.path.insert(0, 'src/data')
sys.path.insert(0, 'src/dgl_graphsage')
sys.path.insert(1, '..')
from api.spotifyAPI import SpotifyAPI
from utils import load_data
from train_updated import train

import dgl
import torch
import torch.nn.functional as F
from dgl import save_graphs
from dgl import load_graphs

from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

sys.path.insert(0, './src/features')
from features.build_features import load_data as graph_from_scratch
import re
from torch.nn.functional import normalize
import matplotlib.pyplot as plt

from itertools import combinations

import seaborn as sns

In [5]:
import recommend as r

# Load Data

In [3]:
# %%time
# # Loading in the data. ~ 00:01:30
# with open('config/data-params.json') as fh:
#     data_cfg = json.load(fh)
# feat_data, adj_list, dgl_G, uri_map = load_data(**data_cfg)

### From Scratch

In [4]:
# uses original files to make the graph from scratch

# def scratch(feat_dir, normal=True):
#     print('Loading feature data...')
#     data = np.genfromtxt(feat_dir, delimiter=',', skip_header=True, dtype=str)
#     features = np.array(np.delete(data[:,2:], -3, 1), dtype=float)
#     if normal:
#         features = normalize(torch.Tensor(features), dim=0)
#     uris = data[:, 1]
#     uris = [re.sub('spotify:track:', '', uri) for uri in uris]
#     uri_map = {n: i for i,n in enumerate(uris)}
    
#     G = graph_from_scratch('./data/a13group1/data/', 'Spotify Playlist', 0, 0, 0)
#     print('graph created')
#     src, dest = [], [] 
#     adj_list = defaultdict(set)    
#     for e in G.edges:
#         u,v = uri_map[e[0]], uri_map[e[1]]
#         adj_list[u].add(v)
#         adj_list[v].add(u)
#         src.append(u)
#         dest.append(v)
        
#     src = torch.tensor(src)
#     dest = torch.tensor(dest)
        
    
#     print('adj list created')
    
#     dgl_G = dgl.graph((src, dest), num_nodes=len(G.nodes))
#     return features, adj_list, dgl_G, uri_map

In [5]:
# '''
# Creates graph from scratch using from_networkx
# results in double edges
# '''
# def scratch(feat_dir, normal=True):
#     print('Loading feature data...')
#     data = np.genfromtxt(feat_dir, delimiter=',', skip_header=True, dtype=str)
#     features = np.array(np.delete(data[:,2:], -3, 1), dtype=float)
#     if normal:
#         features = normalize(torch.Tensor(features), dim=0)
#     uris = data[:, 1]
#     uris = [re.sub('spotify:track:', '', uri) for uri in uris]
#     uri_map = {n: i for i,n in enumerate(uris)}
    
#     G = graph_from_scratch('./data/a13group1/data/', 'Spotify Playlist', 0, 0, 0)
#     print('graph created')
#     src, dest = [], [] 
#     adj_list = defaultdict(set)    
#     for e in G.edges:
#         u,v = uri_map[e[0]], uri_map[e[1]]
#         adj_list[u].add(v)
#         adj_list[v].add(u)
#         src.append(u)
#         dest.append(v)
#     print('adj list created')
    
#     #dgl_G = dgl.graph((src, dest), num_nodes=len(G.nodes))
#     dgl_G = dgl.from_networkx(G)
#     return features, adj_list, dgl_G, uri_map

### Currently the one I am using

In [6]:
'''
Loads the double edged graph
feat_dir = feature directory
'''
def load_double_edge(feat_dir, double_edge_dir, normal=True):
    print('Loading feature data...')
    data = np.genfromtxt(feat_dir, delimiter=',', skip_header=True, dtype=str)
    features = np.array(np.delete(data[:,2:], -3, 1), dtype=float)
    if normal:
        features = normalize(torch.Tensor(features), dim=0)
    uris = data[:, 1]
    uris = [re.sub('spotify:track:', '', uri) for uri in uris]
    uri_map = {n: i for i,n in enumerate(uris)}
    listed = list(uri_map)
    
    G  = load_graphs(double_edge_dir)[0][0]
    print('Loaded DGL Graph')
    sources = G.edges()[0] 
    destinations = G.edges()[1]
    
    src, dest = [], [] 
    adj_list = defaultdict(set)
    for e in range(len((G.edges()[0]))):
        u,v = sources[e].item(), destinations[e].item()
        adj_list[u].add(v)
        adj_list[v].add(u)
        src.append(u)
        dest.append(v)
        
    
    print('adj list created')
    return features, adj_list, G, uri_map


In [7]:
#%%time
#feat_data, adj_list, dgl_G, uri_map = scratch("./data/a13group1/features/merged_features.csv")

In [8]:
# Saving Graph if built from scratch
# save_graphs("./data/a13group1/double_edges_170k.bin", dgl_G)

In [9]:
%%time
# Loading Graph ~ 3min
feat_dir = "../../data/a13group1/features/merged_features.csv"
double_edge_dir = "../../data/a13group1/double_edges_170k.bin"
feat_data, adj_list, dgl_G, uri_map = load_double_edge(feat_dir, double_edge_dir)

Loading feature data...
Loaded DGL Graph
adj list created
CPU times: user 4min 8s, sys: 23.5 s, total: 4min 31s
Wall time: 4min 31s


In [71]:
# with open("uri_recs_170k.json", "w") as uri_json:
#      json.dump(uri_map, uri_json)

In [10]:
# %%time
# dgl_scratch = dgl.from_networkx(scratch_G)

# Train Model

In [11]:
%%time
# Training the Model. GPU ~ 00:00:40
with open('../../config/model-params.json') as fh:
            model_cfg = json.load(fh)
model, pred = train(dgl_G, feat_data, adj_list, **model_cfg)

# Put everything on CPU
model = model.to('cpu')
pred = pred.to('cpu')


Train pos edge: 26413580
Validation pos edge: 518464
Cuda enabled: True

Training starts:
In epoch 1 batch 1, loss: 0.6208469271659851
In epoch 1 batch 6, loss: 0.5380141139030457
In epoch 1 batch 11, loss: 0.4345894753932953
In epoch 1 batch 16, loss: 0.24024872481822968
In epoch 1 batch 21, loss: 0.11041790246963501

Epoch 1 AUC:  0.41798002968071346
In epoch 2 batch 1, loss: 0.11095213145017624
In epoch 2 batch 6, loss: 0.13438409566879272
In epoch 2 batch 11, loss: 0.13351640105247498
In epoch 2 batch 16, loss: 0.11656180769205093
In epoch 2 batch 21, loss: 0.10473253577947617

Epoch 2 AUC:  0.4348110926857368
In epoch 3 batch 1, loss: 0.11381617188453674
In epoch 3 batch 6, loss: 0.11397577822208405
In epoch 3 batch 11, loss: 0.10435086488723755
In epoch 3 batch 16, loss: 0.10930505394935608
In epoch 3 batch 21, loss: 0.10370529443025589

Epoch 3 AUC:  0.45601295671106534
In epoch 4 batch 1, loss: 0.10520198941230774
In epoch 4 batch 6, loss: 0.10570354759693146
In epoch 4 batch 1

In [12]:
# Create Embeddings
z = model(dgl_G, feat_data)

# Create Predictions
# In the form of the strength of the connection between source, destination from dgl_G.edges()
preds = pred(dgl_G, z)

In [13]:
# Create Nearest Neighbors
neigh = NearestNeighbors(n_neighbors=25, radius=0.4)
neigh.fit(feat_data)

NearestNeighbors(n_neighbors=25, radius=0.4)

# Get Playlist

In [14]:
'''
Get a list of eligible slice files (first 10000 playlists)
thelist: list of directories
'''
def get_eligible(thelist):
    eligible = []
    for x in thelist:
        nums = pd.Series(x.strip('mpd.slice.json').split('-')).astype(int)
        if nums[0] <= 9999:
            eligible.append(x)
    return eligible

In [15]:
def get_random_playlist():
    data_path = (os.path.join(os.path.expanduser('~'), '/teams/DSC180A_FA21_A00/a13group1/data/'))
    file_samp = np.random.choice(get_eligible(pd.Series(os.listdir(os.path.join(os.path.expanduser('~'), '/teams/DSC180A_FA21_A00/a13group1/data/')))), replace=True)
    fname = os.path.join(data_path, file_samp)
    with open(fname) as f:
        data = json.load(f)
        item = np.random.choice(data['playlists'])
    print(fname)    
    return item

In [16]:
'''
Gets random playlist
Gets the track names of the original tracks in the playlist
'''
def get_playlist_info(item):
    print('Playlist ID:', item['pid'])
    print('Playlist Length:', len(item['tracks']))
    
    # Get track names---artist
    original_tracks = []
    for i in range(len(item['tracks'])):
        name = item['tracks'][i]['track_name']+'---'+item['tracks'][i]['artist_name']
        original_tracks.append(name)
        
    # Get track uris
    seeds = []
    for i in item['tracks']:
        uri = i['track_uri'].split(':')[-1]
        seeds.append(uri)
        
    return item, original_tracks, seeds

In [17]:
item = get_random_playlist()

/teams/DSC180A_FA21_A00/a13group1/data/mpd.slice.7000-7999.json


In [18]:
item, original_tracks, seeds = get_playlist_info(item)
original_tracks

Playlist ID: 7009
Playlist Length: 20


['Sneakin’---Drake',
 'Gassed Up---Nebu Kiniza',
 'You Was Right---Lil Uzi Vert',
 'Fire Squad - Live---J. Cole',
 'A Tale of 2 Citiez - Live---J. Cole',
 'Deja Vu---J. Cole',
 '20 Joints---Berner',
 'Moves---Big Sean',
 'Look At Me!---XXXTENTACION',
 'Rent Money---Future',
 'No Heart---21 Savage',
 'Dear Mama---2Pac',
 'Neighbors---J. Cole',
 '4 Your Eyez Only---J. Cole',
 'Miss America---J. Cole',
 'Like Toy Soldiers---Eminem',
 'T-Shirt---Migos',
 'Get Your Walk On---Xzibit',
 'X---Xzibit',
 'I Need A Doctor---Dr. Dre']

# Recommend

In [19]:
'''
Creates dictionary of highest scored recommendation (of songs not in playlist) for each song in playlist
seeds: list of track uris from user's playlist
dgl_G: DGL Graph
z: embeddings generated from model
pred: predictor from model
feat_data: matrix of feature data
'''
def recommend(seeds, dgl_G, z, pred, neigh, feat_data, uri_map):

    listed = list(uri_map) #parse through uri map for uri --> integer

    score_dict = defaultdict(dict)
    for s in seeds:
        s = uri_map[s]
        _, candidates = dgl_G.out_edges(s, form='uv')
        s_embed = z[s].unsqueeze(dim=0)
        edge_embeds = [torch.cat([s_embed, z[c.item()].unsqueeze(dim=0)],1) for c in candidates]
        #print('Node Value:', s, 'Possible Recs:', len(edge_embeds))
        edge_embeds = torch.cat(edge_embeds, 0)
        scores = pred.W2(F.relu(pred.W1(edge_embeds))).squeeze(1)
        val = list(zip(candidates.detach().numpy(), scores.detach().numpy()))
        val.sort(key=lambda x:x[1], reverse=True)
        
        # Make sure the song is not already in the playlist
        # score_dict[s] = val[0]
        inc = 0
        while True and inc < len(val):
            if listed[val[inc][0]] not in seeds:
                score_dict[s] = val[inc][0]
                break
            if inc == (len(val) - 1):
                # If no co-occurence, use 5-NN based on features -- COLD START
                # print('Cold Start, Using Feature Data Instead')
                closest = neigh.kneighbors(feat_data[[s]], 25, return_distance=False)[0]
                for i in closest:
                    if listed[i] not in seeds:
                        score_dict[s] = i
                        break
                break
                    
            else:
                inc += 1
                
    # Get uris            
    uri_recs = []
    for i in score_dict.keys():
        cur_uri = listed[score_dict[i]]
        uri_recs.append(cur_uri)
        
    return uri_recs

In [20]:
uri_recs = recommend(seeds, dgl_G, z, pred, neigh, feat_data, uri_map)
uri_recs

['3U9uz6rqqN68P2ZfuYs9RI',
 '7nQFMZ3IbTFeIA8RxRnzBy',
 '6B0FIZm6Xs4qYRyTjsuU0Y',
 '6jalstnZd8hEJuLGFFjL9L',
 '5rSH2YlI18D4mmgP0tQbs9',
 '7dEYcnW1YSBpiKofefCFCf',
 '7y5zXpTN4J7K0PngQYa655',
 '3hxsFtkwEx6rZ310ROjwPD',
 '08azaGQ1YKWvajMDARPSeZ',
 '7dEYcnW1YSBpiKofefCFCf',
 '06qwPtnc9J0a75Z0l6jj8b',
 '0pSaDwrMLM7F82LWJg9v6s',
 '7crrp07AQujRvbOoIGitk8',
 '5XlpERwWF8SuVUL9AghEIh',
 '1WT11QmhZutciEv1NsHt1R',
 '3VHflEWMd4lsoeqfHTFyTK',
 '5XlpERwWF8SuVUL9AghEIh',
 '5eGEc27nnhtmcOh6RC890a',
 '7crrp07AQujRvbOoIGitk8',
 '6B0FIZm6Xs4qYRyTjsuU0Y']

In [21]:
listed = list(uri_map) #parse through uri map for uri --> integer

# Translate Codes with Spotify API

In [21]:
client_id = ''
client_secret = ''

spotify = SpotifyAPI(client_id, client_secret)

In [22]:
def get_data_spotify(query, api, num):
    chunk = api.get_resource(query, 'tracks', 'v1')
    return chunk

In [23]:
def get_rec_names(uri_recs, api, sleep_time):
    rec_track_names = []
    for i in uri_recs:
        one = get_data_spotify(i, api, 1)
        trackname = one['tracks'][0]['name']

        firstartist = one['tracks'][0]['artists'][0]['name']

        the_rec = trackname+'---'+firstartist
        rec_track_names.append(the_rec)


        time.sleep(sleep_time)
    return rec_track_names

In [24]:
%%time
rec_track_names = get_rec_names(uri_recs, spotify, 2)

CPU times: user 562 ms, sys: 58.4 ms, total: 620 ms
Wall time: 43.8 s


In [25]:
original_tracks

['Sneakin’---Drake',
 'Gassed Up---Nebu Kiniza',
 'You Was Right---Lil Uzi Vert',
 'Fire Squad - Live---J. Cole',
 'A Tale of 2 Citiez - Live---J. Cole',
 'Deja Vu---J. Cole',
 '20 Joints---Berner',
 'Moves---Big Sean',
 'Look At Me!---XXXTENTACION',
 'Rent Money---Future',
 'No Heart---21 Savage',
 'Dear Mama---2Pac',
 'Neighbors---J. Cole',
 '4 Your Eyez Only---J. Cole',
 'Miss America---J. Cole',
 'Like Toy Soldiers---Eminem',
 'T-Shirt---Migos',
 'Get Your Walk On---Xzibit',
 'X---Xzibit',
 'I Need A Doctor---Dr. Dre']

In [26]:
pd.Series(rec_track_names).value_counts()

The Truth---Kevin Gates                                                                                3
Bad Meets Evil---Eminem                                                                                3
Ka-Bang! (feat. MF Doom)---CZARFACE                                                                    2
Fade Away (feat. Kevin Gates & Mook Boy)---Bezz Believe                                                1
Inertia---ARTY                                                                                         1
03' Adolescence - Live---J. Cole                                                                       1
Yamborghini High (feat. Juicy J)---A$AP Mob                                                            1
Doowutchyalike---Digital Underground                                                                   1
Rap Up 2016---Uncle Murda                                                                              1
Got It (feat. Snoop Dogg)---Boys Noize                 

Note: We kept repeats because a song that is recommended as the best more than once throughout the playlist should be ranked higher

# Batch Recommendation

### Create json file of recommendations

In [54]:
# %%time
# all_accs = []
# for each in range(50):
#     item = get_random_playlist()
#     item, original_tracks, seeds = get_playlist_info(item)
#     uri_recs = recommend(seeds, dgl_G, z, pred, neigh, feat_data, uri_map)
    
#     checks = []
#     for i in range(len(uri_recs)):
#         well = uri_map[uri_recs[i]] in dgl_G.out_edges(uri_map[seeds[i]])[1]
#         checks.append(well)
        
#     acc = np.count_nonzero(checks) / len(checks)
#     all_accs.append({'pid': item['pid'], 'seeds': seeds, 'recs':uri_recs, 'accuracy': acc})
    
# with open("sample_recommendations.json", "w") as final:
#     json.dump(all_accs, final)

/teams/DSC180A_FA21_A00/a13group1/data/mpd.slice.6000-6999.json
Playlist ID: 6853
Playlist Length: 190
/teams/DSC180A_FA21_A00/a13group1/data/mpd.slice.4000-4999.json
Playlist ID: 4153
Playlist Length: 148
/teams/DSC180A_FA21_A00/a13group1/data/mpd.slice.0-999.json
Playlist ID: 193
Playlist Length: 14
/teams/DSC180A_FA21_A00/a13group1/data/mpd.slice.3000-3999.json
Playlist ID: 3268
Playlist Length: 8
/teams/DSC180A_FA21_A00/a13group1/data/mpd.slice.7000-7999.json
Playlist ID: 7423
Playlist Length: 38
/teams/DSC180A_FA21_A00/a13group1/data/mpd.slice.6000-6999.json
Playlist ID: 6151
Playlist Length: 15
/teams/DSC180A_FA21_A00/a13group1/data/mpd.slice.9000-9999.json
Playlist ID: 9266
Playlist Length: 11
/teams/DSC180A_FA21_A00/a13group1/data/mpd.slice.0-999.json
Playlist ID: 161
Playlist Length: 161
/teams/DSC180A_FA21_A00/a13group1/data/mpd.slice.4000-4999.json
Playlist ID: 4692
Playlist Length: 7
/teams/DSC180A_FA21_A00/a13group1/data/mpd.slice.2000-2999.json
Playlist ID: 2809
Playlist 